# Ground state calculations of the double well quantum dot

Given a basis set $\{\psi_p\}_{p = 1}^L$ of single particle functions that are eigenfunctions to the one-body Hamiltonian operator for the double well system, $h_{\text{DW}}$, i.e.,

\begin{align}
    h_{\text{DW}}\rvert\psi_p\rangle = \epsilon_p\rvert\psi_p\rangle,
\end{align}

where $\epsilon_p$ is the eigenenergy of the single particle function $\rvert\psi_p\rangle$. The double well one-body operator (with a perturbation in the $x$-direction) is given by

\begin{align}
    h_{\text{DW}}
    &= -\frac{\hbar^2}{2m}\nabla^2 + \frac{1}{2}m\omega^2 r^2
        + \frac{1}{2}m\omega^2\left(
            \frac{1}{4}R^2 - R|x|
        \right) \\
    &= h_{\text{HO}} + \frac{1}{2}m\omega^2\left(
            \frac{1}{4}R^2 - R|x|
        \right),
\end{align}
where $h_{\text{HO}}$ is the one-body Hamiltonian for the harmonic oscillator. The one-body potentials are thus

\begin{gather}
    v_{\text{HO}} = \frac{1}{2}m\omega^2 r^2, \\
    v_{\text{DW}} = v_{\text{HO}} + \frac{1}{2}m\omega^2\left(
            \frac{1}{4}R^2 - R|x|
        \right).
\end{gather}

We can solve this directly by discretizing the space we are working in and constructing the full matrix $h_{\text{DW}}$ before diagonalizing.

## Solving the one-dimensional double well quantum dot directly

By limiting ourselves to one dimension we can set up the one-body Hamiltonian matrix directly and diagonalize it to find the eigenenergies, $\epsilon_p$, and the corresponding eigenfunctions, $\rvert\psi_p\rangle$.

In [5]:
import numpy as np

We restrict ourselves to equal masses and natural units, thus we set

\begin{align}
    \hbar = m = c = 1.
\end{align}

In [13]:
# Natural units
m = 1
hbar = 1

Below we set the frequency, $\omega$, and the radius of the double well, $R$.

In [21]:
omega = 1
radius = 2

Next, we set up the potentials.

In [28]:
# Note that r is one-dimensional
v_ho = lambda r, omega, m=m, hbar=hbar: 0.5*m*omega**2*r**2
v_dw = lambda r, omega, radius=radius, m=m, hbar=hbar: v_ho(r, omega, m=m, hbar=hbar) \
    + 0.5*m*omega**2*(0.25*radius**2 - radius*abs(r))

### Setting up the one-body Hamiltonian matrix

We use central finite differences for the kinetic term in $h_{\text{DW}}$.

\begin{align}
    \frac{\text{d}^2 u(x)}{\text{d} x^2}
    \approx \frac{u(x + \Delta x) - 2 u(x) + u(x - \Delta x)}{(\Delta x)^2}.
\end{align}

By letting the wavefunction $u(x)$ work on the one-dimensional one-body Hamiltonian we get

\begin{align}
    h_{\text{DW}}u(x) = -\frac{1}{2}\frac{\text{d}^2 u(x)}{\text{d} x^2} + v_{\text{DW}}u(x).
\end{align}

We now discretize the space into $i \in \{1, n\}$ points and insert the finite difference approximation. We label the wavefunction by $u(x_i) = u_i$.

\begin{align}
    h_{\text{DW}}u_{i}
    &= -\frac{1}{2} \frac{u_{i + 1} - 2 u_i + u_{i - 1}}{(\Delta x)^2}
        + v_{\text{DW}}u_i \\
    &= \left(\frac{1}{(\Delta x)^2} + v_{\text{DW}}\right) u_i
        - \frac{1}{2(\Delta x)^2}\left(u_{i + 1} + u_{i - 1}\right).
\end{align}

By treating $u$ as a vector with components $u_i$ we can construct a matrix from the one-body Hamiltonian.

\begin{align}
    h_{\text{DW}} u
    &=
    \begin{pmatrix}
        \frac{1}{(\Delta x)^2} + v_{\text{DW}} & -\frac{1}{2(\Delta x)^2} & 0 & 0 & \dots & 0 & 0 \\
        -\frac{1}{2(\Delta x)^2} & \frac{1}{(\Delta x)^2} + v_{\text{DW}} & -\frac{1}{2(\Delta x)^2}
        & 0 & \dots & 0 & 0 \\
        0 & -\frac{1}{2(\Delta x)^2} & \frac{1}{(\Delta x)^2} + v_{\text{DW}}
        & -\frac{1}{2(\Delta x)^2} & \dots & 0 & 0 \\
        \vdots & & \ddots & & \dots & & \\
        0 & 0 & 0 & 0 & \dots & -\frac{1}{2(\Delta x)^2}
        & \frac{1}{(\Delta x)^2} + v_{\text{DW}}
    \end{pmatrix}
    \begin{pmatrix}
        u_1 \\
        u_2 \\
        u_3 \\
        u_4 \\
        u_5 \\
        \vdots \\
        u_n
    \end{pmatrix}
\end{align}

In [27]:
# The size of the spatial room we look at
grid_length = -10
# Number of points to discretize the grid in
num_points = 100

In [25]:
x = np.linspace(-grid_length, grid_length, num_points)
delta_x = x[1] - x[0]